In [ ]:
import os
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
RES_PATH = "../res"
DOCS_PATH = os.path.join(RES_PATH, "docs")
WELLNESS_PATH = os.path.join(DOCS_PATH, "Bienestar")
PHYSICAL_HEALTH_PATH = os.path.join(DOCS_PATH, "Salud Fisica")
MENTAL_HEALTH_PATH = os.path.join(DOCS_PATH, "Salud Mental")
ALL_DOCS = os.path.join(DOCS_PATH, "All_docs")

CHROMA_PATH = os.path.join(RES_PATH, "CHROMA_DB")
DATA_PATH = "data"

openai_keys_file = os.path.join(RES_PATH, os.path.join("keys", "openai_key.txt"))

In [ ]:
def extract_keys(file_path):
    keys = {}
    try:
        with open(file_path, 'r') as file:
            for line in file:
                key, value = line.strip().split(':')
                keys[key] = value
    except FileNotFoundError:
        print("El archivo especificado no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo: {e}")
    return keys

# Uso de la función para obtener las claves
keys = extract_keys(openai_keys_file)

# Accediendo a las variables
key_name = keys.get('name')
key_secret = keys.get('secret')

PROMPT

In [ ]:
PROMPT_TEMPLATE = """Basándote exclusivamente en el siguiente contexto:
{context}
---
Resuelve la siguiente cuestión: {question}
"""

In [ ]:
embedding_function = OpenAIEmbeddings(openai_api_key=key_secret)
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [ ]:
def query_rag(query_text: str, db):

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    # if len(results) == 0 or results[0][1] < 0.7:
    #     print(f"Unable to find matching results!")
    #     return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    model = ChatOpenAI(openai_api_key=key_secret, model="gpt-3.5-turbo")
    answer = model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]

    topic = model.predict(f"Considerando estas tres temáticas: Bienestar, Salud mental y Salud física, necesito que se \
                                  devuelva únicamente de resultado la temática a la que crees que pertenece el siguiente texto: {answer}")

    return (answer, sources, topic)

In [ ]:
text = "mi amiga sara se ha tomado 20 chupitos de jagger y esta tirada en el suelo inconsciente. que puedo hacer"

In [ ]:
answer, sources, topic = query_rag(text, db)

In [ ]:
answer

In [ ]:
sources

In [ ]:
topic